<a href="https://colab.research.google.com/github/ParkEunHyeok/AI_Study/blob/main/NLP/Document_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install konlpy
!pip3 install jamo

     |████████████████████████████████| 19.4 MB 7.9 MB/s 
     |████████████████████████████████| 448 kB 36.3 MB/s 
     |████████████████████████████████| 86 kB 5.2 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [2]:
import requests # 파파고 번역 api 사용
import re
import jpype
import requests
from gensim.models import Word2Vec
from jamo import h2j, j2hcj
from konlpy.tag import Okt

In [3]:
# 구글 드라이브 연결
import os
from google.colab import drive
drive.mount('/content/gdrive/')
path = "gdrive/My Drive/Colab Notebooks/PMS/"

Mounted at /content/gdrive/


In [4]:
# 파일 확장자 및 특수문자 제거 후 형태소 명사 단위로 분석

def morpheme(file_name):
  okt = Okt()

  no_extension_file_name = file_name.rsplit('.', 1)[0] 
  cleaned_file_name = re.sub('[-=+,_#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]',
                             '', no_extension_file_name)

  return cleaned_file_name

In [5]:
# 영어 문장 한글로 번역

def get_translate_ko(text):
    client_id = "fSuwQC7wzKdKWD4f0Gbl" # <-- client_id 기입
    client_secret = "upCLLCyDRI" # <-- client_secret 기입

    data = {'text' : text,
            'source' : 'en',
            'target': 'ko'}

    url = "https://openapi.naver.com/v1/papago/n2mt"

    header = {"X-Naver-Client-Id":client_id,
              "X-Naver-Client-Secret":client_secret}

    response = requests.post(url, headers=header, data=data)
    rescode = response.status_code

    if(rescode==200):
        send_data = response.json()
        trans_data = (send_data['message']['result']['translatedText'])
        return trans_data
    else:
        print("Error Code:" , rescode)

In [6]:
# 한글 문장 영어로 번역

def get_translate_eng(text):
    client_id = "fSuwQC7wzKdKWD4f0Gbl" # <-- client_id 기입
    client_secret = "upCLLCyDRI" # <-- client_secret 기입

    data = {'text' : text,
            'source' : 'ko',
            'target': 'en'}

    url = "https://openapi.naver.com/v1/papago/n2mt"

    header = {"X-Naver-Client-Id":client_id,
              "X-Naver-Client-Secret":client_secret}

    response = requests.post(url, headers=header, data=data)
    rescode = response.status_code

    if(rescode==200):
        send_data = response.json()
        trans_data = (send_data['message']['result']['translatedText'])
        return trans_data
    else:
        print("Error Code:" , rescode)

In [23]:
# 데이터 전처리 함수

def data_preprocess(data_list): # 파일 리스트 / 디렉토리 리스트
    preprocessed_list = data_list
    tmp_data=""
    result = []

    # 형태소 분석, 특수기호 제거
    for data_index in range(len(preprocessed_list)):
        split_data = morpheme(preprocessed_list[data_index])
        preprocessed_list[data_index] = split_data
    
    # 번역 api : 영어 번역, 띄어쓰기
    for index in range(len(preprocessed_list)):
      # 영어 거쳐서 한국어 번역하면 정확도 높은 대신
      # 시간 오래걸리고 api 일일 사용량 제한 더 잘걸림
      eng_list = get_translate_eng(preprocessed_list[index])
      ko_list = get_translate_ko(eng_list)
      # ko_list = get_translate_ko(preprocessed_list[index])
      result.append(ko_list)
    return result

In [8]:
# 폴더 안에 들어간 파일들의 내용 저장

def save_classificated_file(index_dict, file_list, directory_list):
    result_dict = {}

    for key, value in index_dict.items():
        result_dict[file_list[key]] = directory_list[value]

    return result_dict

In [9]:
# 매칭된 파일 제거

def rm_classificated_file(index_dict,file_list):  # 매칭된 파일 제거
    data_name = []
    for index in list(index_dict.keys()):
        data_name.append(file_list[index])

    file_list =  [x for x in file_list if x not in (data_name)]

    return file_list

In [10]:
# 단순 유사도 검사
# 파일명과 카테고리명이 같으면 그 폴더 안에 넣어줌

def simplest_classification(directory_list, file_list):  # 단어 유사도 검사 => index return해서 제거하는 방법을 쓴다.
    tmp_dict = {}
    if jpype.isJVMStarted():
      jpype.attachThreadToJVM()
    okt = Okt()
    classificated_dir_index = 0
    for file_index in range(len(file_list)):
        file_morphs = okt.morphs(file_list[file_index])
        
        max_weight = 0
        for dir_index in range(len(directory_list)): # 디렉토리 갯수 : 3
            dir_morphs = okt.morphs(directory_list[dir_index])
            tmp_weight = 0
            for file_element in file_morphs: # 형태분석한 파일 갯수
                for dir_element in dir_morphs:
                    
                    if (dir_element == file_element) and (check_format(dir_element) == ('kor')):
                        tmp_weight += 2      
                    	#한글로 되있을 경우 가중치 2
                    elif ((file_element in dir_element) and (check_format(dir_element) == ('num'))):
                        tmp_weight += 1
                        #영어로 되있을 경우 가중치 1
                        
            if (tmp_weight > max_weight):
                max_weight = tmp_weight
                classificated_dir_index = dir_index
        if max_weight > 0:
            tmp_dict[file_index] = classificated_dir_index

    return tmp_dict

In [11]:
# 한글과 숫자를 판단하고 가중치를 넣어주는 함수
# 숫자보다 한글의 가중치가 높게 설정

def check_format(input): # 한국어 , 영어 , 숫자 형태 분석
    value = input[0]
    if ord('가') <= ord(value) <= ord('힣'):
        return 'kor'
    elif value.isdigit():
        return 'num'
    elif value.isalpha():
        return 'eng'
    else:
        return False

In [12]:
# Word2Vec 유사도 측정
# 제목과 카테고리명 사이의 유사도를 측정, 퍼센테이지 출력

def word2vec_similarity(directory_list, file_list):
    model = Word2Vec.load(path+'ko.bin')

    tmp_dict = {}
    classificated_dir_index = 0
    if jpype.isJVMStarted():
      jpype.attachThreadToJVM()
    okt = Okt() # 형태소 분석
    for file_index in range(len(file_list)):
        max_similarity = 0
        file_morphs = okt.morphs(file_list[file_index])
        for dir_index in range(len(directory_list)):
            dir_morphs = okt.morphs(directory_list[dir_index])
            tmp_similarity = 0
            for file_element in file_morphs:
                for dir_element in dir_morphs:
                    if check_format(file_element) == 'kor' and check_format(dir_element) == 'kor':
                        try:
                            tmp_similarity = model.similarity(file_element, dir_element)
                        except KeyError:
                            pass
                if tmp_similarity > max_similarity:  # 유사도 수치는 사용자가 설정
                    max_similarity = tmp_similarity
                    classificated_dir_index = dir_index

            tmp_dict[file_index] = classificated_dir_index

    return tmp_dict

In [13]:
# 자모 단위를 이용한 유사도 검사

def Phonology_classification(directory_list, file_list):
    trans_file_jumo = []
    trans_folder_jumo = []
    equl_count = [[0 for i in range(len(file_list))] for i in range(len(directory_list))]
    result_dict = {}

    for i in file_list:
        a = j2hcj(h2j(i))
        line = []
        for j in a:
            line.append(j2hcj(h2j(j)))
        trans_file_jumo.append(line)

    for i in directory_list:
        a = j2hcj(h2j(i))
        line = []
        for j in a:
            line.append(j2hcj(h2j(j)))
        trans_folder_jumo.append(line)

    for i in range(len(directory_list)):
        for j in range(len(file_list)):
            for k in trans_file_jumo[j]:
                if k in trans_folder_jumo[i]:
                    equl_count[i][j] += 1

    for i in range(len(directory_list)):
        for j in range(len(file_list)):
            if len(trans_folder_jumo[i]) <= equl_count[i][j]:
                result_dict[j] = i

    return result_dict

In [14]:
def dict_to_list(dict):
    result = []

    for key, value in dict.items():
        tmp = str(value) + "/"+ str(key)
        result.append(tmp)

    return result

In [15]:
# 기타 폴더
# 분류가 안된 파일들을 기타로 분류해줌

def except_directory(file_list):
    out_of_classification = "except_directory"
    result_dict = {}

    for file in file_list:
        result_dict[file] = out_of_classification

    return result_dict

In [16]:
# 분류될 카테고리명
directory_list = ['넷마블', '대통령', '민주당', '모모랜드', '레드벨벳', '빅히트', '소프트', '코스피', '텔레콤', '트롯', '포토']
# 분류할 문서 이름
file_list = ['코스피외국인개인매수에코앞까지코스닥돌파.txt', '더불어민주당미래입법과제상임위간사단연석회의.txt', '자급제아이폰분실보험가입일부터가능.txt', '모모랜드주이자가격리붐대신스페셜등장어마어마하게떨려붐붐파워.txt', '넷마블월드시즌챕터업데이트.txt', '코스피사흘연속사상최고치경신.txt', '민주당미래입법과제상임위간사단연석회의.txt', '텔레콤맵안드로이드오토베타테스트론칭.txt', '놀면뭐하니톱귀유재석추억의주크박스오픈만여명과깜짝라이브.txt', '발라드그룹순순희신곡전부다주지말걸발표.txt', '트롯신홍원빈테스형으로열정무대패자부활전탈락에도극찬.txt', '넷마블월드새로운챕터성카드추가.txt', '코스피는오르고환율은내리고.txt', '자급제아이폰써도분실보험가입가능해진다.txt', '장동민배다해선넘는사생활피해에곤혹이슈톡.txt', '코스피사상최고치경신.txt', '다시뒤집힌전동킥보드법무면허규제법행안위통과.txt', '빅히트레이블즈가지테마로펼쳐진다.txt', '자급제아이폰일부터분실보험가입가능.txt', '코스피사상최고치.txt', '텔레콤자급제아이폰도분실보험가입된다.txt', '소프트웨이브디지털전환사이버보안이핵심.txt', '이낙연공수처법반드시완수많이인내했고결단임박.txt', '세일즈포스고급차브랜드벤틀리에솔루션공급.txt', '코스피다시역대최고가마감.txt', '모모랜드주이붐붐파워대타상큼미모.txt', '코스피종가기준최고치경신.txt', '미래입법과제상임위간사단연석회의참석하는이낙연김태년.txt', '대통령절차적정당성공정성당부추미애징계위연기.txt', '레드벨벳아이린오랜만에근황화보촬영스타.txt', '코스피마감사흘연속사상최고치경신.txt', '코스피일째사상최고원달러환율원대.txt', '소프트웨이브플렌옵틱콘텐츠저작기술.txt', '트롯신이떴다홍원빈테스형으로레전드무대선사.txt', '미래입법과제발언하는이낙연대표.txt', '코스피일연속최고치경신선목전서마감종합.txt', '코스피마감또최고치경신선눈앞.txt', '코스피거침없는질주오르며사흘째최고치경신.txt', '민주당국회자리에아파트짓자는건토건포퓰리즘.txt', '포토하하별후딱끝내고갈게.txt', '표시장거래현황.txt', '크루미디어지콤위시트레이닝센터업무제휴계약체결.txt', '민주당미래입법과제연석회의.txt', '대통령절차적정당공정성강조에징계위일로연기.txt', '코스피하루만에최고치경신.txt', '윤보미김민경비장함감도는마녀들포스터운동신들케미에기대.txt', '발언하는이낙연대표.txt', '대통령이용구징계위원장배제지시했지만공정성논쟁은여전.txt', '미래입법과제상임위간사단연석회의.txt', '자급제아이폰고객도분실보험가입된다.txt', '모모랜드낸시나윤한복전도사뿜뿜.txt', '대화하는이낙연한정애.txt', '텔레콤빅테크마케팅컴퍼니로조직개편.txt', '레드벨벳아이린갑질논란후첫근황미모에물올랐네스타.txt', '코스피최고치경신눈앞삼성전자장중만원첫돌파.txt', '소프트웨이브모아소프트등분석시험도구공개.txt', '빅히트뉴이어스이브라이브.txt', '목요대화참석자와인사하는정세균총리.txt', '코스피사상최고치원달러환율원붕괴.txt', '소프트웨이브혁신기술선보인다스타트업총출동.txt', '소프트웨이브플랫폼부터의료진단시스템까지신기술총출동.txt', '자급제아이폰도일부터분실보험가입가능.txt', '김태희싸이다이아몬드수저스타상위집안연중라이브.txt', '목소리내는여당전의원들추미애가검찰개혁어렵게해.txt', '자급제아이폰분실보험가입월일부터가능.txt', '발언하는김태년원내대표.txt', '코스피일째사상최고치.txt', '코스피사흘연속사상최고치경신선눈앞.txt', '트롯전국체전트로트전쟁서막알리는예고편공개.txt', '코스피역대최고가로마감.txt', '트롯전국체전레전드라인업과함께트로트시대연다.txt', '빅히트패밀리콘서트에서도본다새해카운트다운무대공개.txt', '자급제아이폰도분실보험가능.txt', '민주당권력기관개혁마무리수순야당공수처결사반대.txt', '소프트웨이브국내대표솔루션업계차세대먹거리로미래준비한다.txt', '코스피일째사상최고선코앞.txt', '스브스타남자로살겠다선언한엘리엇페이지에배우자도지지자랑스러워.txt', '단독박명수이찬원밀접접촉자개뼈다귀녹화연기.txt', 'untitled.ui', '민주국민의힘김은혜대변인즉각적인사과촉구.txt', '미래입법과제연석회의참석하는이낙연김태년.txt']

In [20]:
# 제목 기반 자동 분류
def title_classification(directory_list, file_list):
    result_dict = {}
    index_dict = {}

    origin_directory_list = directory_list.copy()
    origin_file_list = file_list.copy()

    # 데이터 전처리
    tmp_dir_list = data_preprocess(directory_list)
    tmp_file_list = data_preprocess(file_list)

    # 단순 유사도 검사
    index_dict = simplest_classification(tmp_dir_list, tmp_file_list)
    ## update
    result_dict.update(save_classificated_file(index_dict, origin_file_list, origin_directory_list))
    tmp_file_list = rm_classificated_file(index_dict, tmp_file_list)
    origin_file_list = rm_classificated_file(index_dict, origin_file_list)

    # Word2Vec를 이용한 유사도 검사
    index_dict = word2vec_similarity(tmp_dir_list, tmp_file_list)
    ## update
    result_dict.update(save_classificated_file(index_dict, origin_file_list, origin_directory_list))
    tmp_file_list = rm_classificated_file(index_dict, tmp_file_list)
    origin_file_list = rm_classificated_file(index_dict, origin_file_list)

    # 자모단위 유사도 검사
    index_dict = Phonology_classification(tmp_dir_list, tmp_file_list)
    ## update
    result_dict.update(save_classificated_file(index_dict, origin_file_list, origin_directory_list))
    tmp_file_list = rm_classificated_file(index_dict, tmp_file_list)
    origin_file_list = rm_classificated_file(index_dict, origin_file_list)

    # 기타 분류
    result_dict.update(except_directory(origin_file_list))

    result_list = dict_to_list(result_dict)
    return result_list

In [21]:
res = title_classification(directory_list, file_list)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).


In [22]:
for i in res:
  print(i, end="\n")

넷마블/코스피외국인개인매수에코앞까지코스닥돌파
넷마블/더불어민주당미래입법과제상임위간사단연석회의
넷마블/자급제아이폰분실보험가입일부터가능
넷마블/모모랜드주이자가격리붐대신스페셜등장어마어마하게떨려붐붐파워
넷마블/넷마블월드시즌챕터업데이트
넷마블/코스피사흘연속사상최고치경신
넷마블/민주당미래입법과제상임위간사단연석회의
넷마블/텔레콤맵안드로이드오토베타테스트론칭
넷마블/놀면뭐하니톱귀유재석추억의주크박스오픈만여명과깜짝라이브
넷마블/발라드그룹순순희신곡전부다주지말걸발표
넷마블/트롯신홍원빈테스형으로열정무대패자부활전탈락에도극찬
넷마블/넷마블월드새로운챕터성카드추가
넷마블/코스피는오르고환율은내리고
넷마블/자급제아이폰써도분실보험가입가능해진다
넷마블/장동민배다해선넘는사생활피해에곤혹이슈톡
넷마블/코스피사상최고치경신
넷마블/다시뒤집힌전동킥보드법무면허규제법행안위통과
넷마블/빅히트레이블즈가지테마로펼쳐진다
넷마블/자급제아이폰일부터분실보험가입가능
넷마블/코스피사상최고치
넷마블/텔레콤자